In [ ]:
from model import CondInvicuna

import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
from viz import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
config = {'num_bijectors': 6,
          'bijector': 'RealNVP',
          'network_size': 64,
          'use_batchnorm': True,
          'y_dim': 2,
          'x_dim': 1,
          'nn_layers': [64,64,32],
          'activation': 'tanh',
          'sigma_eps': 0.01,
          'batch_size': 20,
          'data_horizon': 2,
          'test_horizon': 50,
          'learning_rate': 1e-4}

In [ ]:
class MultistepDataset:
    def __init__(self, aux_dim=0, noise_var=0.001, rng=None):
        self.step_min = -3.
        self.step_max = 3.
        self.num_steps = 3
        self.x_range = [-5.,5.]
        self.noise_std = np.sqrt( noise_var )
        
        self.x_dim = 1
        self.aux_dim = aux_dim
        self.y_dim = 1 + aux_dim
            
        self.np_random = rng
        if rng is None:
            self.np_random = np.random
            
    def sample(self, n_funcs, n_samples, return_lists=False):
        x = np.zeros((n_funcs, n_samples, self.x_dim))
        y = np.zeros((n_funcs, n_samples, self.y_dim))
        
        step_mat = np.zeros((n_funcs, self.num_steps))
        
        for i in range(n_funcs):
            step_pts = self.step_min + self.np_random.rand(self.num_steps)* (self.step_max - self.step_min)
            step_mat[i,:] = step_pts
            
            x_samp = self.x_range[0] + self.np_random.rand(n_samples)*(self.x_range[1] - self.x_range[0])
            y_samp = self.multistep(x_samp, step_pts)

            x[i,:,0] = x_samp
            y[i,:,0] = y_samp
            y[i,:,1:] = self.np_random.randn(n_samples, self.aux_dim)

        if return_lists:
            return x,y,step_mat

        return x,y
    
    def multistep(self, x, step_pts):
        x = x.reshape([1,-1])
        step_pts = step_pts.reshape([-1,1])
        y = 2.*np.logical_xor.reduce( x > step_pts, axis=0) - 1.
        y += self.noise_std*self.np_random.randn(x.shape[1])
        return y

In [ ]:
g1 = tf.Graph()
sess1 = tf.Session(config=tf.ConfigProto(log_device_placement=True), graph=g1)

In [ ]:
model = CondInvicuna(sess1, config)
model.construct_model()

In [ ]:
dataset = MultistepDataset(1)

In [ ]:
xx,yy = dataset.sample(10,100)
for j in range(1):
    plt.scatter(xx[j,:,0],yy[j,:,0])
plt.savefig('cond_meta_sample.pdf')
plt.show()

In [ ]:
model.train(dataset,5000)

In [ ]:
x_q = np.linspace(-5,5,50).reshape([1,-1,1])
x_t,y_t = dataset.sample(1,50)

n_samples = 10
n_context = 4

#print(x_t[:,:n_context,:].shape)
# y_sample = model.gen_samples(x_t[:,:n_context,:],y_t[:,:n_context,:],x_q,n_samples)

In [ ]:
xx = x_t.reshape([-1, 1])
yy = y_t[:,:,0].reshape([-1,1])

In [ ]:
plt.scatter(xx,yy)

In [ ]:
for i in range(n_samples):
    plt.plot(x_q[0,:,0],y_sample[i,0,:,0])
plt.show()

In [ ]:
plot_conditional_likelihood(model, x_t[:,:0,:],y_t[:,:0,:], num_aux=1)
plt.savefig('conditional_prior_noflow.pdf')
plt.show()

In [ ]:
plot_conditional_likelihood(model, x_t[:,:50,:],y_t[:,:50,:], num_aux=1)
plt.savefig('conditional_posterior_noflow.pdf')
plt.show()